# **Übungsaufgaben DDL**

Dieser Übungsblock enthält Aufgaben zum Thema _Datenbankschema (DDL)_. Löse die Aufgaben in der gestellten Reihenfolge.

## CREATE

**1.1** Damit die Kunden in einer Bestellung mehrere Produkte bestellen können, ist in der Datenbank (Schema _verkauf_) eine weitere Tabelle nötig. Erstelle hierfür die Tabelle _bestellpositionen_ mit den folgenden Attributen in der vorgegebenen Reihenfolge: 
- _id_: Primärschlüsselspalte, exakter numerischer Wert, Auto-Inkrement
- _bestellid_: exakter numerischer Wert, darf nicht leer sein
- _produktid_: exakter numerischer Wert, darf nicht leer sein
- _menge_: exakter numerischer Wert, darf nicht leer sein

In [2]:
-- Stelle sicher, dass du dich in der richtigen Datenbank befindest!
USE m106;
CREATE TABLE verkauf.bestellpositionen (
    id INT PRIMARY KEY IDENTITY(1,1),
    bestellid INT NOT NULL,
    produktid INT NOT NULL,
    menge SMALLINT NOT NULL
);

Commands completed successfully.

Total execution time: 00:00:00.023

**1.2** Für die Speicherung von sensitiven Daten erstellst du ein neues Schema mit dem Namen _intern_.

In [3]:
CREATE SCHEMA intern;

Commands completed successfully.

Total execution time: 00:00:00.001

**1.3** In dem neu erstellten Schema erstellst du nun die Tabelle _mitarbeiter_ gemäss dieser Definition:
- _mitarbeiterid_: Primärschlüsselspalte, exakt numerischer Wert, Auto-Inkrement 
- _vorname_: Character String, kann leer sein
- _nachname_: Character String, darf nicht leer sein
- _geschlecht_: Character String, darf nicht leer sein und nur die Werte _m_, _w_, _n_ enthalten
- _geburtsdatum_: Datum, darf nicht leer sein
- _anstellungsdatum_: Datum, darf leer sein
- _kader_: Boolean Datentyp, darf leer sein

In [4]:
CREATE TABLE intern.mitarbeiter (
    mitarbeiterid INT PRIMARY KEY IDENTITY(1,1),
    vorname VARCHAR(50) NULL,
    nachname VARCHAR(50) NOT NULL,
    geschlecht CHAR(1) CHECK (geschlecht in ('m','w','n')),
    geburtsdatum DATE NOT NULL,
    anstellungsdatum DATE,
    kader BIT
);

Commands completed successfully.

Total execution time: 00:00:00.004

In [5]:
-- anschliessende Kontrolle, ob die Tabelle wie definiert in der DB existiert
EXEC sp_columns @table_name = N'mitarbeiter',  @table_owner = N'intern'; 

(7 rows affected)

Total execution time: 00:00:00.022

TABLE_QUALIFIER,TABLE_OWNER,TABLE_NAME,COLUMN_NAME,DATA_TYPE,TYPE_NAME,PRECISION,LENGTH,SCALE,RADIX,NULLABLE,REMARKS,COLUMN_DEF,SQL_DATA_TYPE,SQL_DATETIME_SUB,CHAR_OCTET_LENGTH,ORDINAL_POSITION,IS_NULLABLE,SS_DATA_TYPE
m106,intern,mitarbeiter,mitarbeiterid,4,int identity,10,4,0,10,0,NULL,NULL,4,NULL,NULL,1,NO,56
m106,intern,mitarbeiter,vorname,12,varchar,50,50,NULL,NULL,1,NULL,NULL,12,NULL,50,2,YES,39
m106,intern,mitarbeiter,nachname,12,varchar,50,50,NULL,NULL,0,NULL,NULL,12,NULL,50,3,NO,39
m106,intern,mitarbeiter,geschlecht,1,char,1,1,NULL,NULL,1,NULL,NULL,1,NULL,1,4,YES,39
m106,intern,mitarbeiter,geburtsdatum,-9,date,10,20,NULL,NULL,0,NULL,NULL,-9,NULL,NULL,5,NO,0
m106,intern,mitarbeiter,anstellungsdatum,-9,date,10,20,NULL,NULL,1,NULL,NULL,-9,NULL,NULL,6,YES,0
m106,intern,mitarbeiter,kader,-7,bit,1,1,NULL,NULL,1,NULL,NULL,-7,NULL,NULL,7,YES,50


**1.4** Um den Mitarbeitern Löhne auszahlen zu können, wird eine Tabelle namens _lohnkonto_ benötigt, in welcher die folgenden Informationen gespeichert werden sollen (Achtung: es geht hier um sensitive Daten):
- _kontoid_: Primärschlüsselspalte, exakt numerischer Wert, Auto-Inkrement
- _iban_: Character String, darf nicht leer sein
- _mitarbeiterid_: exakt numerischer Wert, darf nicht leer sein, Fremdschlüssel zu _intern.mitarbeiter.mitarbeiterid_
- _gueltigab_: Datum, darf nicht leer sein
- _gueltigbis_: Datum, darf leer sein

In [6]:
CREATE TABLE intern.lohnkonto (
    kontoid INT PRIMARY KEY IDENTITY(1,1),
    iban VARCHAR(20) NOT NULL,
    mitarbeiterid INT NOT NULL FOREIGN KEY REFERENCES intern.mitarbeiter(mitarbeiterid),
    gueltigab DATE NOT NULL,
    gueltigbis DATE
);

Commands completed successfully.

Total execution time: 00:00:00.004

## ALTER

**2.1** Erweitere die Tabelle _bestellpositionen_ um die neue Spalte _rabatt_ (exakt numerischer Wert).

In [7]:
ALTER TABLE verkauf.bestellpositionen
ADD rabatt TINYINT;

Commands completed successfully.

Total execution time: 00:00:00.004

**2.2** Stelle sicher, dass in der Mitarbeiter-Tabelle auch Vor- und Nachnamen mit Zeichen aus dem Unicode-Zeichensatz gespeichert werden können. 

In [8]:
ALTER TABLE intern.mitarbeiter
ALTER COLUMN vorname NVARCHAR(50) NULL;

ALTER TABLE intern.mitarbeiter
ALTER COLUMN nachname NVARCHAR(50) NOT NULL;

Commands completed successfully.

Total execution time: 00:00:00.010

**2.3** Füge der Mitarbeiter-Tabelle auch gleich noch das neue Feld _email_ hinzu. Bestimme selbst, welcher Datentyp hier der geeignetste ist.

In [9]:
ALTER TABLE intern.mitarbeiter
ADD email VARCHAR(50) NULL;

Commands completed successfully.

Total execution time: 00:00:00.003

**2.4** Das Attribut _kader_ ist in der Tabelle _intern.mitarbeiter_ fehl am Platz. Bitte entferne dieses aus der Tabelle.

In [10]:
ALTER TABLE intern.mitarbeiter
DROP COLUMN kader;

Commands completed successfully.

Total execution time: 00:00:00.003

## Schlüsselfelder und Constraints

**3.1** Der Tabelle _verkauf.bestellpositionen_ fehlen noch die Fremdschlüssel. Überlege dir, welche das sind und füge sie hinzu.

In [11]:
ALTER TABLE verkauf.bestellpositionen
ADD FOREIGN KEY (bestellid) REFERENCES verkauf.bestellungen(bestellid);

ALTER TABLE verkauf.bestellpositionen
ADD CONSTRAINT fk_bestellpos_produktid FOREIGN KEY (produktid) REFERENCES verkauf.produkte(produktid);

Commands completed successfully.

Total execution time: 00:00:00.004

**3.2** Bei der Tabelle _lohnkonto_ soll auf der Spalte _gueltigbis_ der Standard-Wert '31.12.9999' definiert werden. Bitte nimm diese Änderung vor.

In [12]:
ALTER TABLE intern.lohnkonto
ADD DEFAULT '31.12.9999' FOR gueltigbis;

Commands completed successfully.

Total execution time: 00:00:00.003